# 0 Configuration

In [ ]:
config = {
    "feature_eng": {

    },
    "modeling": {
        "dummy": False,
        "linear_reg": False,
        "svr": False,
        "tree": False,
        "forest": False,
        "xgboost": False
    }
}

***
# 1 Dependency import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import mutual_info_regression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import set_config

import xgboost
from xgboost import XGBRegressor

import shap

# plt.style.use(["dark_background"])
plt.style.use(["default"])
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

np.random.seed(0)

In [ ]:
def get_categorical_features_name(dataset, split_by_unique_count=True, split_count=10):
    features_name = dataset.select_dtypes(["object", "bool"]).columns
    if split_by_unique_count:
        less_uniques = [feature_name for feature_name in features_name if dataset[feature_name].nunique() <= split_count]
        lot_uniques = features_name.difference(less_uniques).tolist()
        return (less_uniques, lot_uniques)
    else:
        return features_name.values

In [ ]:
def get_numerical_features_name(dataset):
    features_name = dataset.select_dtypes(["int64", "float64"]).columns.values.tolist()
    return features_name

In [ ]:
class EvaluationResult():
    def __init__(self, gridsearch, target, X_train, y_train, X_test, y_test, num_cols, cat_less_unique_cols, cat_lot_unique_cols):
        self.__gs = gridsearch
        self.__X_train = X_train
        self.__y_train = y_train
        self.__X_test = X_test
        self.__y_test = y_test
        self.__target = target
        self.__score = self.__gs.score(self.__X_test, self.__y_test)
        self.__ratio = np.abs((self.__score * 100) / self.__y_test.mean())
        self.__num_cols = num_cols
        self.__cat_less_unique_cols = cat_less_unique_cols
        self.__cat_lot_unique_cols = cat_lot_unique_cols
        self.__cat_less_unique_cols_preproc = self.__gs.best_estimator_["transforms"].transformers_[1][1]["one_hot_encoder"].get_feature_names_out(self.__cat_less_unique_cols).tolist()
        self.__cat_lot_unique_cols_preproc = self.__gs.best_estimator_["transforms"].transformers_[2][1]["one_hot_encoder"].get_feature_names_out(self.__cat_lot_unique_cols).tolist()

        self.best_model = self.__gs.best_estimator_.named_steps["model"]
        self.feature_names = self.__num_cols + self.__cat_less_unique_cols_preproc + self.__cat_lot_unique_cols_preproc
        self.X_train_transform = pd.DataFrame(data=self.__gs.best_estimator_.named_steps["transforms"].transform(self.__X_train).toarray(), columns=self.feature_names)
        self.X_test_transform = pd.DataFrame(data=self.__gs.best_estimator_.named_steps["transforms"].transform(self.__X_test).toarray(), columns=self.feature_names)

    def print_metrics(self):
        print(f"RMSE: {-self.__score:.4}")
        print(f"Target mean value: {self.__target.mean():.4}")
        print(f"Ratio: {self.__ratio:.4}%")
        print(f"best_params: {self.__gs.best_params_}")

In [ ]:
def evaluate(model, grid_params, dataset, target, scoring="neg_root_mean_squared_error"):
    set_config(display="diagram") # display="text" -> for textual output

    ### DATASET PREPARATION ###

    y = dataset[target].copy()
    X = dataset.drop(columns=[target]).copy()

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

    categorical_cols_less_unique, categorical_cols_lot_unique = get_categorical_features_name(X_train)
    numerical_cols = get_numerical_features_name(X_train)

    ### PIPELINE CONSTRUCTION ###

    num_pipe = Pipeline(steps=[
        ("simple_imputer", SimpleImputer(strategy="mean"))
    ])

    cat_less_unique_pipe = Pipeline(steps=[
        ("simple_imputer", SimpleImputer(strategy="most_frequent")),
        ("one_hot_encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    cat_lot_unique_pipe = Pipeline(steps=[
        ("simple_imputer", SimpleImputer(strategy="most_frequent")),
        ("one_hot_encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer([
        ("num_pipe", num_pipe, numerical_cols),
        ("cat_less_unique_pipe", cat_less_unique_pipe, categorical_cols_less_unique),
        ("cat_lot_unique_pipe", cat_lot_unique_pipe, categorical_cols_lot_unique)
    ])

    pipeline = Pipeline([
        ("transforms", preprocessor),
        ("model", model)
    ])

    ### GRIDSEARCH DECLARATION AND FITTING ###

    gs = GridSearchCV(pipeline, grid_params, scoring=scoring, refit=True)
    gs.fit(X_train, y_train)

    return EvaluationResult(gs, y, X_train, y_train, X_test, y_test, numerical_cols, categorical_cols_less_unique, categorical_cols_lot_unique)

In [ ]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

***
# 2 Loading data

In [ ]:
data = pd.read_csv("data/data-cleaned.csv", delimiter=",")

In [ ]:
data.drop(columns=["Unnamed: 0"], inplace=True)

In [ ]:
target_1 = "SiteEnergyUse(kBtu)"
target_2 = "GHGEmissionsIntensity(kgCO2e/ft2)"

In [ ]:
# y = data[target]
# X = data.drop(columns=[target])

In [ ]:
# X_train_full, X_test_full, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

In [ ]:
# categorical_cols_less_unique = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]
# categorical_cols_lot_unique = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() >= 10 and X_train_full[cname].dtype == "object"]
# numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [ ]:
# my_cols = categorical_cols_lot_unique + categorical_cols_less_unique + numerical_cols
# X_train = X_train_full[my_cols].copy()
# X_test = X_test_full[my_cols].copy()

In [ ]:
# y_test.describe()

***
# 3 Feature engineering

## 3.1 Mutual Information

In [ ]:
# X_MI = X.copy()

In [ ]:
# X_MI["DefaultData"] = X_MI["DefaultData"].astype("object")

In [ ]:
# X_MI_NUM = X_MI.select_dtypes(["int64", "float64"]).columns
# X_MI_CAT = X_MI.select_dtypes(["object"]).columns

In [ ]:
# discrete_features = X_MI.dtypes == object

In [ ]:
# impt_num = SimpleImputer(strategy="mean")
# X_MI[X_MI_NUM] = pd.DataFrame(impt_num.fit_transform(X_MI[X_MI_NUM]))

In [ ]:
# impt_cat = SimpleImputer(strategy="most_frequent")
# X_MI[X_MI_CAT] = pd.DataFrame(impt_cat.fit_transform(X_MI[X_MI_CAT]))

In [ ]:
# for col in X_MI_CAT:
#     X_MI[col], _ = X_MI[col].factorize()

In [ ]:
# scores = make_mi_scores(X_MI, y, discrete_features).sort_values(ascending=False)
# scores.head()

In [ ]:
# def plot_mi_scores(scores):
#     plt.figure(dpi=100, figsize=(8, 5))
#     scores = scores.sort_values(ascending=True)
#     width = np.arange(len(scores))
#     ticks = list(scores.index)
#     plt.barh(width, scores)
#     plt.yticks(width, ticks)
#     plt.title("Mutual Information Scores")

# plot_mi_scores(scores)

***
# 4 Modeling

***
## 4.1 DummyRegressor

In [ ]:
%%time
if config["modeling"]["dummy"]:

    model = DummyRegressor()

    grid_params = [
        {
            "model__strategy": ["mean", "median"]
        },
        {
            "model__strategy": ["quantile"],
            "model__quantile": np.arange(0, 1.1, 0.1),
        }
    ]

    score, ratio, gs, pipeline = evaluate(model, grid_params)

    print(f"RMSE: {score:n} / RATIO: {ratio:.4}%")
    print(f"best_params: {gs.best_params_}")

***
## 4.2 LinearRegression

In [ ]:
%%time
if config["modeling"]["linear_reg"]:

    model = LinearRegression()

    grid_params = [
        {
            "model__fit_intercept": [True]
        }
    ]

    score, ratio, gs, pipeline = evaluate(model, grid_params)

    print(f"RMSE: {score:n} / RATIO: {ratio:.4}%")
    print(f"best_params: {gs.best_params_}")

***
## 4.3 SupportVectorRegression

In [ ]:
%%time
if config["modeling"]["svr"]:

    model = SVR()

    grid_params = [
        {
            "model__kernel": ["rbf"],
            "model__degree": [3],
            "model__gamma": ["scale"]
        }
    ]

    score, ratio, gs, pipeline = evaluate(model, grid_params)

    print(f"RMSE: {score:n} / RATIO: {ratio:.4}%")
    print(f"best_params: {gs.best_params_}")

***
## 4.4 DecisionTreeRegressor

In [ ]:
%%time
if config["modeling"]["tree"]:

    model = DecisionTreeRegressor()

    grid_params = [
        {
            "model__random_state": [1],
            "model__max_depth": [2, 3, 4],
            "model__min_samples_leaf": range(1, 11, 1),
            "model__criterion": ["mse"]
        }
    ]

    score, ratio, gs, pipeline = evaluate(model, grid_params)

    print(f"RMSE: {score:n} / RATIO: {ratio:.4}%")
    print(f"best_params: {gs.best_params_}")

***
## 4.5 RandomForestRegressor

In [ ]:
%%time
if config["modeling"]["forest"]:

    model = RandomForestRegressor()

    grid_params = [
        {
            "model__random_state": [1],
            "model__n_estimators": [25],
            "model__min_samples_leaf": [1],
            "model__criterion": ["mse"]
        }
    ]

    score, ratio, gs, pipeline = evaluate(model, grid_params)

    print(f"RMSE: {score:n} / RATIO: {ratio:.4}%")
    print(f"best_params: {gs.best_params_}")

***
## 4.6 XGBRegressor

In [ ]:
%%time
if config["modeling"]["xgboost"]:

    model = XGBRegressor()

    grid_params = [
        {
            "model__random_state": [1],
            "model__max_depth": [3],
            "model__n_estimators": [100],
            "model__learning_rate": [0.1],
        }
    ]

    score, ratio, pipeline, gs, X_train, y_train, X_test, y_test, categorical_cols_less_unique, categorical_cols_lot_unique = evaluate(model, grid_params, data, target_1)

    print(f"RMSE: {-score:.4}")
    print(f"Target mean value: {y_train.mean():.4}")
    print(f"Ratio: {ratio:.4}%")
    print(f"best_params: {gs.best_params_}")

***
## 4.7 Permutation, PDP, SHAP

In [ ]:
model = XGBRegressor()

grid_params = [
    {
        "model__random_state": [1],
        "model__max_depth": [3],
        "model__n_estimators": [100],
        "model__learning_rate": [0.1],
    }
]

score, ratio, target, gs, best_model, feature_names = evaluate(model, grid_params, data, target_1)

print(f"RMSE: {-score:.4}")
print(f"Target mean value: {target.mean():.4}")
print(f"Ratio: {ratio:.4}%")
print(f"best_params: {gs.best_params_}")

***
### 4.6.3 Feature explainability

In [ ]:
explainer = shap.TreeExplainer(eval_result.best_model, data=eval_result.X_train_transform)
shap_values = explainer(eval_result.X_test_transform)

In [ ]:
shap.plots.waterfall(shap_values[0])

In [ ]:
shap.plots.force(shap_values[0])

In [ ]:
# shap.force_plot(explainer.expected_value, shap_values.values) # takes too much time

In [ ]:
shap.plots.scatter(shap_values[:,"Address_14027 lake city way ne"], color=shap_values)

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
shap.plots.bar(shap_values)

***
# 5 ...

...

***
# 6 ...